In [1]:
import sys

import torch
from collections import OrderedDict

import numpy as np
import matplotlib.pyplot as plt
import scipy.io
from scipy.interpolate import griddata
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
import warnings

warnings.filterwarnings('ignore')

np.random.seed(1234)

In [2]:
# CUDA support 
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

In [3]:
# the deep neural network
class DNN(torch.nn.Module):
    def __init__(self, layers):
        super(DNN, self).__init__()
        
        # parameters
        self.depth = len(layers) - 1
        
        # set up layer order dict
        self.activation = torch.nn.Tanh
        
        layer_list = list()
        for i in range(self.depth - 1): 
            layer_list.append(
                ('layer_%d' % i, torch.nn.Linear(layers[i], layers[i+1]))
            )
            layer_list.append(('activation_%d' % i, self.activation()))
            
        layer_list.append(
            ('layer_%d' % (self.depth - 1), torch.nn.Linear(layers[-2], layers[-1]))
        )
        layerDict = OrderedDict(layer_list)
        
        # deploy layers
        self.layers = torch.nn.Sequential(layerDict)
        
    def forward(self, x):
        out = self.layers(x)
        return out

In [4]:
# the physics-guided neural network
class PhysicsInformedNN():
    def __init__(self, X, u, layers, lb, ub):
        
        # boundary conditions
        self.lb = torch.tensor(lb).float().to(device)
        self.ub = torch.tensor(ub).float().to(device)
        
        # data
        self.x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        self.t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)
        self.u = torch.tensor(u).float().to(device)
        
        # settings
        self.lambda_1 = torch.tensor([0.1], requires_grad=True).to(device)
        self.lambda_2 = torch.tensor([30.0], requires_grad=True).to(device)
        self.lambda_3 = torch.tensor([30.0], requires_grad=True).to(device)
        self.lambda_4 = torch.tensor([30.0], requires_grad=True).to(device)
        self.lambda_5 = torch.tensor([100.0], requires_grad=True).to(device)
        
        self.lambda_1 = torch.nn.Parameter(self.lambda_1)
        self.lambda_2 = torch.nn.Parameter(self.lambda_2)
        self.lambda_3 = torch.nn.Parameter(self.lambda_3)
        self.lambda_4 = torch.nn.Parameter(self.lambda_4)
        self.lambda_5 = torch.nn.Parameter(self.lambda_5)
        
        # deep neural networks
        self.dnn = DNN(layers).to(device)
        self.dnn.register_parameter('lambda_1', self.lambda_1)
        self.dnn.register_parameter('lambda_2', self.lambda_2)
        self.dnn.register_parameter('lambda_3', self.lambda_3)
        self.dnn.register_parameter('lambda_4', self.lambda_4)
        self.dnn.register_parameter('lambda_5', self.lambda_5)
        
         # optimizers: using the same settings
        self.optimizer = torch.optim.LBFGS(
            self.dnn.parameters(), 
            lr=1, 
            max_iter=50000, 
            max_eval=50000, 
            history_size=50,
            tolerance_grad=1e-9, 
            tolerance_change=1.0*np.finfo(float).eps,
            line_search_fn="strong_wolfe"       # can be "strong_wolfe"
        )
        
        self.optimizer_Adam = torch.optim.Adam(self.dnn.parameters())
        self.iter = 0
        
    def net_u(self, x, t):  
        lambda_1 = self.lambda_1 
        lambda_2 = self.lambda_2
        lambda_3 = self.lambda_3
        u_temp = self.dnn(torch.cat([x, t], dim=1))
        u_temp_t = torch.autograd.grad(
            u_temp, t, 
            grad_outputs=torch.ones_like(u_temp),
            retain_graph=True,
            create_graph=True
        )[0]
        
        u_temp_tt = torch.autograd.grad(
            u_temp_t, t, 
            grad_outputs=torch.ones_like(u_temp_t),
            retain_graph=True,
            create_graph=True
        )[0]
        
        u = lambda_3 * u_temp_tt + lambda_2 * u_temp_t + 0.6681489 * u_temp
        return u
    
    def net_f(self, x, t):
        """ The pytorch autograd version of calculating residual """
        lambda_5 = self.lambda_5
        lambda_4 = self.lambda_4
        u_temp = self.dnn(torch.cat([x, t], dim=1))
        
        u_temp_t = torch.autograd.grad(
            u_temp, t, 
            grad_outputs=torch.ones_like(u_temp),
            retain_graph=True,
            create_graph=True
        )[0]
        
        u_temp_tt = torch.autograd.grad(
            u_temp_t, t, 
            grad_outputs=torch.ones_like(u_temp_t),
            retain_graph=True,
            create_graph=True
        )[0]
        
        f = lambda_5 * u_temp_tt + lambda_4 * u_temp_t + u_temp - x
        return f
    
    def loss_func(self):
        u_pred = self.net_u(self.x, self.t)
        f_pred = self.net_f(self.x, self.t)
        loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
        self.optimizer.zero_grad()
        loss.backward()
        
        self.iter += 1
        if self.iter % 100 == 0:
            print(
                'Loss: %e, l2: %.6f, l3: %.6f, l4: %.6f, l5: %.6f' % 
                (
                    loss.item(), 
                    # self.lambda_1.item(), 
                    self.lambda_2.item(),
                    self.lambda_3.item(),
                    self.lambda_4.item(),
                    self.lambda_5.item()
                )
            )
        return loss
    
    def train(self, nIter):
        self.dnn.train()
        for epoch in range(nIter):
            u_pred = self.net_u(self.x, self.t)
            f_pred = self.net_f(self.x, self.t)
            loss = torch.mean((self.u - u_pred) ** 2) + torch.mean(f_pred ** 2)
            
            # Backward and optimize
            self.optimizer_Adam.zero_grad()
            loss.backward()
            self.optimizer_Adam.step()
            
            if epoch % 100 == 0:
                print(
                    'It: %d, Loss: %.3e, Lambda_2: %.6f, Lambda_3: %.6f, Lambda_4: %.6f, Lambda_5: %.6f' % 
                    (
                        epoch, 
                        loss.item(), 
                        # self.lambda_1.item(), 
                        self.lambda_2.item(),
                        self.lambda_3.item(),
                        self.lambda_4.item(),
                        self.lambda_5.item()
                    )
                )
                
        # Backward and optimize
        self.optimizer.step(self.loss_func)
    
    def predict(self, X):
        x = torch.tensor(X[:, 0:1], requires_grad=True).float().to(device)
        t = torch.tensor(X[:, 1:2], requires_grad=True).float().to(device)

        self.dnn.eval()
        u = self.net_u(x, t)
        f = self.net_f(x, t)
        u = u.detach().cpu().numpy()
        f = f.detach().cpu().numpy()
        return u, f

In [5]:
# nu = 0.01/np.pi

N_u = 1000000
layers = [2, 20, 20, 20, 1]

data = scipy.io.loadmat('data/SMR_test_100.mat')

t = data['t'].flatten()[:,None]
x = data['x'].flatten()[:,None]
Exact = np.real(data['usol']).T

X, T = np.meshgrid(x, t)

X_star = np.hstack((X.flatten()[:,None], T.flatten()[:,None]))
u_star = Exact.flatten()[:,None]              

# Doman bounds
lb = X_star.min(0)
ub = X_star.max(0) 

In [6]:
%%time

noise = 0.0            

# create training set
idx = np.random.choice(X_star.shape[0], N_u, replace=False)
X_u_train = X_star[idx,:]
u_train = u_star[idx,:]

# X_u_train = X_star
# u_train = u_star

# training
model = PhysicsInformedNN(X_u_train, u_train, layers, lb, ub)
# model.train(0)
model.train(55000)

It: 0, Loss: 5.164e+03, Lambda_2: 29.999001, Lambda_3: 30.000999, Lambda_4: 29.999001, Lambda_5: 100.000999
It: 100, Loss: 4.629e+03, Lambda_2: 29.880014, Lambda_3: 29.994986, Lambda_4: 29.880512, Lambda_5: 99.988022
It: 200, Loss: 4.234e+03, Lambda_2: 29.855083, Lambda_3: 29.974131, Lambda_4: 29.855497, Lambda_5: 99.982040
It: 300, Loss: 3.946e+03, Lambda_2: 29.849249, Lambda_3: 29.982103, Lambda_4: 29.850641, Lambda_5: 100.020874
It: 400, Loss: 3.694e+03, Lambda_2: 29.844446, Lambda_3: 30.005781, Lambda_4: 29.851036, Lambda_5: 100.083244
It: 500, Loss: 3.466e+03, Lambda_2: 29.834845, Lambda_3: 30.056530, Lambda_4: 29.856001, Lambda_5: 100.175034
It: 600, Loss: 3.256e+03, Lambda_2: 29.816423, Lambda_3: 30.134432, Lambda_4: 29.868816, Lambda_5: 100.284492
It: 700, Loss: 3.061e+03, Lambda_2: 29.784388, Lambda_3: 30.245180, Lambda_4: 29.893963, Lambda_5: 100.401093
It: 800, Loss: 2.879e+03, Lambda_2: 29.728165, Lambda_3: 30.384928, Lambda_4: 29.932619, Lambda_5: 100.509102
It: 900, Loss: